In [ ]:
from pathlib import Path


datapath = Path("data")

# Dados tipo raster

<div class="alert-info">
    
### Perguntas
- O que são dados raster?
- Quais são os principais atributos de um dado raster?
- Onde obter esse tipo de dados e como recortar uma regioão de interesse?

### Objetivos
- Entender o modelo de dados raster.
- Descrever as vantages e desvantages de armazenar dados em raster.
- Encontrar, recortar e exibir as imagens. 
  
</div>

## Estrutura de Dados: Raster e Vetor

- Vetores e raster são os dois tipo fundamentais de dados.

- Vetores representam feições específicas na superfície da Terra, e designa atributos à essas feições.

- Dados raster são armazenados em grades que são renderizados em um mapa na forma de pixels. Cada pixel representa uma área na supefície da Terra. 


### Raster

Um raster é uma grade retungular de pixels com valores que podem ser contínuos (ex. elevação) ou categórico (ex. uso de terra).

Curiosidade: nos anos 1950's gráficos raster eram famosos por ser uma alternativa a vetores mais rápida e barata (mas de baixa resolução).

### Propriedades

- estrutura de linhas e colunas
- tipo de dado (dtype ou _bit depth_) -- ex 8-bit ou 0-255
- alguma informação de resolução, em geral dpi (dots per inch)

Um raster raster geospacial difere de uma foto digital em um metadado extra sobre o sistema referenciamento de coordenadas.

<img src="images/raster_concept.png" width="500" height="500" />
*Source: National Ecological Observatory Network (NEON)*

## Raster Vantagens and Desvantagens

| Vantagens                              | Desvantagens                                                                  |
|----------------------------------------|-------------------------------------------------------------------------------| 
| representação de superfícies contíguas | tamanho de arquivos muito grande                                              | 
| auto nível de detalhes                 | pode ser difícil representar dados complexos                                  |
| dados são "sem peso" em sua extensão   | Dados são representados em grade regular, o que pode não reflete o mundo real |  
| computação célula-por-célula é ágil    | Modelos espaciais assumem que todos os pixels tem valores                     |
|                                        | Mudança na resolução mudam drasticamente o significado do set de dados        |


In [ ]:
import ipyleaflet

lon, lat = -48.548889, -27.596944
dx = dy = 0.25
bbox = [lon-dx, lat+dy, lon+dx, lat-dy]
west, north, east, south = bbox


m = ipyleaflet.Map(center=[lat, lon], zoom=6)

right_layer = ipyleaflet.basemap_to_tiles(
    ipyleaflet.basemaps.NASAGIBS.ModisTerraTrueColorCR, "2004-03-27"
#     ipyleaflet.basemaps.NASAGIBS.ModisAquaTrueColorCR, "2004-03-27"
)

left_layer = ipyleaflet.TileLayer()
control = ipyleaflet.SplitMapControl(
    left_layer=left_layer, right_layer=right_layer
)
m.add_control(control)

m

### Não existe computação geospacial sem o GDAL

O que significa que `pip` não será o suficiente.

![](images/gdal-meme.png)

In [ ]:
!gdalinfo --version

In [ ]:
!gdalinfo --formats

### Área de interesse

In [ ]:
lon, lat = -48.548889, -27.596944
dx = dy = 0.25

bbox = [lon-dx, lat+dy, lon+dx, lat-dy]
west, north, east, south = bbox

In [ ]:
import json

aoi = {
    "type": "Polygon",
    "coordinates": [
        [
            [west, south],
            [west, north],
            [east, north],
            [east, south],
            [west, south]
        ]
    ]
}


with open('aoi-floripa.geojson', 'w') as f:
    json.dump(aoi, f)


print(json.dumps(aoi, sort_keys=False, indent=2))

In [ ]:
from satsearch import Search


bbox = [-48.798889, -27.846944, -48.298889, -27.346944]

search = Search(bbox=bbox)
print(f"Achei {search.found()} images")

search = Search(time="2019-08-23T00:00:00Z/2019-08-28T00:00:00Z")
print(f"com {search.found()} itens temporais")

In [ ]:
search = Search(
    bbox=bbox,
    datetime="2019-08-23/2019-08-28",
    property=["eo:cloud_cover<5"]
)

items = search.items()
print(items._collections)
print(items.summary())

In [ ]:
items.save("items-floripa.json")

### O que temos nesse JSON?

In [ ]:
import geopandas


gdf = geopandas.read_file("items-floripa.json")
gdf = gdf.sort_values("datetime").reset_index(drop=True)
gdf.head()

<div class="alert-warning">

Exercício:

1. Usando o `repr` do Jupyter mostre um item da coluna `geometry`.
2. O que temos na coluna `eo:bands`?

</div>

```python
gdf[""]
```

In [ ]:
import ast
import pandas as pd


band_info = pd.DataFrame(
    ast.literal_eval(gdf.iloc[0]['eo:bands'])
)
band_info

In [ ]:
subset = gdf[gdf["eo:cloud_cover"] < 20]

print(f"{len(subset)} items")

### Área de Interesse (AOI)

In [ ]:
import hvplot.pandas
import geoviews

gaoi = geopandas.read_file("aoi-floripa.geojson")

cols = gdf.loc[:, ("id","geometry")]

footprints = cols.hvplot(
    geo=True,
    line_color="k",
    alpha=0.1,
)

aoi = gaoi.hvplot(geo=True, line_color="r", fill_color=None)
tiles = geoviews.tile_sources.CartoEco.options(width=700, height=500)
labels = geoviews.tile_sources.StamenLabels.options(level="annotation")
tiles * footprints * aoi * labels

In [ ]:
from ipywidgets import interact
from IPython.display import display, Image


def browse_images(items):
    n = len(items)
    print(n)

    def view_image(i=0):
        item = items[i]
        print(f"id={item.id}\tdate={item.datetime}\tcloud%={item['eo:cloud_cover']}")
        display(Image(item.asset("thumbnail")["href"]))
    
    interact(view_image, i=(0,n-1))

In [ ]:
results = Search(
    bbox=bbox,
    datetime="2019-08-23/2019-08-28",
    property=["eo:cloud_cover<15"]
)

print("%s items" % results.found())
items = results.items()

In [ ]:
browse_images(items)

### Variáveis para melhorar a eficiência de leitura de dados no AWS S3

In [ ]:
import rasterio


env = rasterio.Env(
    GDAL_DISABLE_READDIR_ON_OPEN="EMPTY_DIR",
    CPL_VSIL_CURL_ALLOWED_EXTENSIONS="TIF",
)

In [ ]:
import xarray as xr


item = items[2]
band = 'red'
url = item.asset(band)['href']
print(url)
with env:
    with rasterio.open(url) as src:
        print(src.profile) # image metadata
        width = src.width
        blockx = src.profile['blockxsize']
        blocky = src.profile['blockysize']
        xchunk = int(width/blockx)*blockx
        ychunk = blocky
        da = xr.open_rasterio(src, chunks={'band': 1, 'x': xchunk, 'y': ychunk})
da

### Um dos `repr` mais legais que já vi!

In [ ]:
da.data

In [ ]:
import hvplot.xarray


img = da.hvplot.image(
    rasterize=True,
    logz=True,
    width=700,
    height=500,
    cmap="reds",
    title=f"{item.id} ({band})")

img

In [ ]:
import cartopy.crs as ccrs


crs = ccrs.UTM(zone="22J")
img = da.hvplot.image(
    crs=crs,
    rasterize=True,
    width=700,
    height=500,
    cmap="reds",
    alpha=0.8,
    title=f"{item.id} ({band})")

aoi = gaoi.hvplot(
    geo=True,
    line_color="b",
    fill_color=None,
)
img * aoi

### Cortar a Área de Interesse

In [ ]:
gaoi["geometry"].squeeze()

In [ ]:
from rasterio.mask import mask


with rasterio.open(url) as src:
    # re-project vector to match raster CRS
    print(src.meta)
    shape = gaoi.to_crs(epsg=src.crs.to_epsg())
    out_image, out_transform = mask(
        src, shape.geometry.values, crop=True
    )
    out_meta = src.meta
    out_meta.update({
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})
    print(out_meta)
    
    # write small image to local Geotiff file
    with rasterio.open("subset.tif", "w", **out_meta) as dst:
        dst.write(out_image)

In [ ]:
import pyproj


lat, lon = -27.512964, -48.4529687

conv = pyproj.Proj("+proj=utm +zone=22J, +south +ellps=WGS84 +datum=WGS84 +units=m +no_defs")

x, y = conv(lon, lat)
x, y

In [ ]:
import rasterio.plot
import matplotlib.pyplot as plt


fig, ax = plt.subplots(figsize=(9, 9))
ax.plot(x, y, "bo")

with rasterio.open("subset.tif") as src:
    rasterio.plot.show(
        src,
        cmap="Reds",
    )

ax.plot()

<div class="alert-warning">

Exercício:

1. Modifique a busca acima para uma nova AOI e cria um mapa com os retângulos das imagens.

2. Caso tenha encontrado imagem landsat carregue ela com o xarray.

</div>

# Plano B caso a internet não aguente!

In [ ]:
# https://registry.opendata.aws/landsat-8/

HOST = "https://landsat-pds.s3.amazonaws.com/c1/L8/042/034/LC08_L1TP_042034_20170616_20170629_01_T1"
IMAGE = "LC08_L1TP_042034_20170616_20170629_01_T1_B4.TIF"

In [ ]:
!gdalinfo /vsicurl/{HOST}/{IMAGE}

In [ ]:
%%bash
HOST='http://landsat-pds.s3.amazonaws.com/c1/L8/042/034/LC08_L1TP_042034_20170616_20170629_01_T1'
IMAGE='LC08_L1TP_042034_20170616_20170629_01_T1_B4.TIF'
gdal_translate -of VRT /vsicurl/$HOST/$IMAGE LC08_L1TP_042034_20170616_20170629_01_T1_B4.vrt

In [ ]:
!gdalinfo LC08_L1TP_042034_20170616_20170629_01_T1_B4.vrt | grep PROJCS

In [ ]:
import rasterio
from rasterio import plot


fname = datapath.joinpath('world.rgb.tif')

world = rasterio.open(fname)

type(world)

In [ ]:
world.meta

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt


fig, (axr, axg, axb) = plt.subplots(1,3, figsize=(12, 4), sharex=True, sharey=True)
plot.show((world, 1), ax=axr, cmap='Reds', title='red channel')
plot.show((world, 2), ax=axg, cmap='Greens', title='green channel')
plot.show((world, 3), ax=axb, cmap='Blues', title='blue channel');

In [ ]:
plot.show_hist(world, bins=50, lw=0.0, stacked=False, alpha=0.3, 
               histtype='stepfilled', title='World Histogram overlaid')


In [ ]:
arr = world.read()

red = arr[0, ...]

In [ ]:
import cartopy
import cartopy.crs as ccrs


fig, ax = plt.subplots(
    figsize=(12, 10),
    subplot_kw={'projection': ccrs.InterruptedGoodeHomolosine()})
ax.set_global()
ax.imshow(red, origin='upper', transform=ccrs.PlateCarree(),
          interpolation=None)
ax.coastlines()
ax.add_feature(cartopy.feature.BORDERS);

<div class="alert-success">

Dados raster são, hoje em dia, pesados e difícil de lidar, mas Python, xarray, e catalogs facilitam a sua exploração, obtenção, manipulação e visualização.

* Raster são dados pixelated em forma de matrix
* Imagens raster são em geral "arrays" de várias dimensões (RGB).
* Raster geospaciais trazem um sistema de referência de coordenadas.
* "Resolution" é o tamanho pixel no "chão", mas especificamente é a habilidade do sensor de identificar objetos no solo.

</div>